<a href="https://colab.research.google.com/github/msamwelmollel/Evaluate-Liger/blob/main/Evaluate_Liger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install liger-kernel
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import os
import torch
import time
import transformers
from datasets import load_dataset
from google.colab import userdata

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')

!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer
from liger_kernel.transformers import apply_liger_kernel_to_llama, apply_liger_kernel_to_gemma

In [ ]:
model_name = "sartifyllc/sartify_gemma2-2B-16bit"

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adding this line automatically monkey-patches the model with the optimized Liger kernels
# apply_liger_kernel_to_llama()
model = model.to('cuda')
apply_liger_kernel_to_gemma()

config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.3k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
alpaca_prompt = """Hapo chini kuna maelezo ya kazi, pamoja na maelezo ya ziada yanayotoa muktadha zaidi. Andika jibu ambalo linakamilisha ombi hilo ipasavyo.

### Maelezo:
{}

### Ziada:
{}

### Jibu:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    questions = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        question = alpaca_prompt.format(instruction, input, "")
        questions.append(question)
    return { "questions" : questions, }
pass



dataset = load_dataset("Mollel/alpaca-swahili", split = "train").select(range(0,30))
# selected_dataset = dataset.select(range(0,10))
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
inputs = []
for i in range(len(dataset)):
  inputs.append(tokenizer([dataset["questions"][i]], return_tensors = "pt").to("cuda"))

In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# Initialize variables for total tokens and time
total_generated_tokens = 0
total_time = 0

# Start time measurement
for i in range(len(dataset)):
    start_time = time.time()
    outputs = model.generate(**inputs[i], max_new_tokens=4000, use_cache=True)
    end_time = time.time()

    # Calculate number of generated tokens for this iteration
    num_generated_tokens = outputs.size(-1) - inputs[i]['input_ids'].size(-1)
    total_generated_tokens += num_generated_tokens

    # Add time taken for this iteration
    total_time += (end_time - start_time)

# Calculate throughput (tokens per second)
throughput = total_generated_tokens / total_time

# Measure peak memory used
peak_memory = torch.cuda.max_memory_reserved()

# Convert peak memory to MB
peak_memory_mb = peak_memory / (1024 ** 2)

# Output the results
print(f"Total generated tokens: {total_generated_tokens}")
print(f"Total time taken: {total_time:.2f} seconds")
print(f"Throughput (tokens per second): {throughput:.2f}")
print(f"Peak memory reserved (MB): {peak_memory_mb:.2f}")

Throughput (tokens per second): 17.78
Peak memory reserved (MB): 10326.00
Response: <bos>Hapo chini kuna maelezo ya kazi, pamoja na maelezo ya ziada yanayotoa muktadha zaidi. Andika jibu ambalo linakamilisha ombi hilo ipasavyo.

### Maelezo:
Toa madokezo matatu ya kudumisha afya njema.

### Ziada:


### Jibu:
1. Kula chakula chenye usawaziko: Kula chakula chenye usawaziko ni muhimu kwa kudumisha afya nzuri. Kula matunda, mboga mboga, nafaka kamili, protini nyepesi, na mafuta yenye afya kunaweza kusaidia kudumisha uzito mzuri, kupunguza hatari ya magonjwa sugu, na kuboresha afya ya jumla. 2. Fanya mazoezi kwa ukawaida: Shughuli za kawaida za kimwili ni muhimu kwa kudumisha afya nzuri. Lengo la kufanya mazoezi kwa angalau dakika 150 kwa wiki, au dakika 75 kwa wiki kwa watu wazima wa umri wa miaka 65 na zaidi. Shughuli hizi zinaweza kujumuisha kutembea, kukimbia, kuogelea, au kuendesha baiskeli. 3. Kulala vya kutosha: Kulala vya kutosha ni muhimu kwa afya nzuri. Watu wanapaswa kulala anga